#Football Player Prediction

In [3]:
#Standard-Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Modellierung
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [5]:
# CSV-Dateien laden
wyscout_df = pd.read_csv("wyscout_players_final.csv")
kaggle_df = pd.read_csv("kaggle_players_final.csv")

# Zieldefinition:
kaggle_df["is_pro"] = 1
wyscout_df["is_pro"] = 1

# Kombinieren
combined_df = pd.concat([kaggle_df, wyscout_df], ignore_index=True)

# Vorschau
combined_df.head()

,name,minutes_played,goals,assists,passes_total,passes_success,shots_total,shots_on_target,duels_total,duels_won,ball_losses,recoveries,is_pro
0,Max Aarons,2881.0,NaN,NaN,45.0,34.0,NaN,0.06,2.16,1.16,NaN,5.53,1
1,Yunis Abdelhamid,2983.0,NaN,NaN,47.0,38.7,NaN,0.18,1.87,1.39,NaN,6.77,1
2,Salis Abdul Samed,2462.0,NaN,NaN,61.0,55.9,NaN,0.18,2.01,1.24,NaN,8.76,1
3,Laurent Abergel,2956.0,NaN,NaN,49.8,40.7,NaN,0.21,3.57,2.23,NaN,8.87,1
4,Charles Abi,45.0,NaN,NaN,12.0,4.0,NaN,0.00,0.00,0.00,NaN,4.00,1


In [6]:
# Feature: Passquote
combined_df["pass_accuracy"] = (
    combined_df["passes_success"] / combined_df["passes_total"]
).round(2)

In [7]:
# Feature: Gesamteffektivität
combined_df["activity_score"] = (
    combined_df["passes_success"].fillna(0) +
    combined_df["duels_won"].fillna(0) +
    combined_df["recoveries"].fillna(0)
).round(2)